In [ ]:
import platform

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

import matplotlib
from matplotlib import pyplot as plt

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

print(f"""
Python: {platform.python_version()} 
Tensorflow: {tf.__version__}
Numpy: {np.__version__}
Matplotlib: {matplotlib.__version__}
""")


# Knowledge Distillation
### Model compression method in which a small model is trained to mimic a pre-trained, larger model
- Typically has a teacher model and a student model

![KD Model Image](https://nni.readthedocs.io/en/latest/_images/distill.png)




# Defining a teacher model
![Teacher Model](teacher.png)

| Layer (type) 	| Output Shape        	| Param # 	|
|--------------	|---------------------	|---------	|
| Conv2D       	| (None, 14, 14, 256) 	| 2560    	|
| LeakyReLU    	| (None, 14, 14, 256) 	| 0       	|
| MaxPooling2D 	| (None, 14, 14, 256) 	| 0       	|
| Conv2D       	| (None, 7, 7, 512)   	| 1180160 	|
| Flatten      	| (None, 25088)       	| 0       	|
| Dense        	| (None, 10)          	| 250890  	|


In [ ]:
# Create the teacher
teacher = keras.Sequential([
        keras.Input(shape=(28, 28, 1)),
    
    ],
    name="teacher",
)

# Defining a student model
![Student Model](student.png)

| Layer (type) 	| Output Shape        	| Param # 	|
|--------------	|---------------------	|---------	|
| Conv2D       	| (None, 14, 14, 16) 	| 160    	|
| LeakyReLU    	| (None, 14, 14, 16) 	| 0       	|
| MaxPooling2D 	| (None, 14, 14, 16) 	| 0       	|
| Conv2D       	| (None, 7, 7, 32)   	| 4640 		|
| Flatten      	| (None, 1568)       	| 0       	|
| Dense        	| (None, 10)          	| 15690  	|

In [ ]:
# Create the student
student = keras.Sequential([
    
    ],
    name="student",
)
# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

# Distiller


## Distiller trains student model using teachers output and hard label
![KD Model Image](KDLoss.png)

## Lets take a look at the loss Fns

### There are two losses

- #### student_loss
    - Student loss is a simple loss function comparing predictions and output
    - SparseCategoricalLoss https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
- #### distillation_loss
    - Distillation loss is a Kullback-Leibler(KL) divergence between teacher's prediction and student's prediction
    - https://www.tensorflow.org/api_docs/python/tf/keras/losses/KLDivergence
    - Softmax & Temperature (T) Scaling factor
       $$ \sigma(z,T) = \frac{e^{z_i / T} }{ \Sigma_j e^{z_j / T} }  $$
        - Smoothes output (smaller inputs have bigger output and big inputs have smaller output)

![Temperature](https://www.researchgate.net/publication/325016605/figure/fig1/AS:635391270977537@1528500492783/An-example-of-categorical-probability-distributions-of-high-temperature-softmax-output.png)

### Combination of the losses

$$ 
\text{total_loss} = 
\alpha \cdot \text{student_loss(} 
        \hat{y}_{student}, y 
    \text{ )} + 
(1-\alpha) \cdot \text{distillation_loss(} 
        \sigma(\hat{y}_{student},T), \sigma(\hat{y}_{teacher},T) 
    \text{)} 
$$

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(self,optimizer, metrics, student_loss_fn,distillation_loss_fn, alpha=0.1, temperature=3):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            
            ### TODO Start ###

            
            
            
            
            ### TODO END ###

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

# Preparing MNIST Dataset

![MNIST Dataset](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

# Training Teacher Model
## Teacher model is usually pretrained / robust 

### Since MNIST dataset is easy to predict, teacher model will only train for 1 epoch

In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=1, use_multiprocessing=True)
teacher.evaluate(x_test, y_test)

# Distiller / Training Student Model
- ## We have a strong teacher model capable of predicting MNIST
- ## Instantiate distiller and train student model

In [ ]:
# Initialize and compile distiller

### TODO START ###
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(

    
    
)
### TODO END ###

# Distill teacher to student
student_history = distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

In [ ]:
# Train student as done usually
student_scratch.compile (
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch_history = student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

In [ ]:
xlabels=range(1,4)
plt.plot(xlabels, student_history.history['sparse_categorical_accuracy'])
plt.plot(xlabels, student_scratch_history.history['sparse_categorical_accuracy'])
plt.title('Accuracy comparison')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.xticks([1,2,3])
plt.legend(['Student', 'Student_scratch'], loc='upper left')
plt.show()